In [25]:
import pandas as pd
from tensorflow.keras.layers import Dense,Bidirectional,LSTM,Embedding,Input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [8]:
from keras.datasets import imdb

In [38]:
data = pd.read_csv("IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [39]:
data.replace({"positive":1,"negative":0},inplace=True)

In [40]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [41]:
X = data.iloc[:,0].values
y = data.iloc[:,1].values

In [55]:
data['sentiment'].index

RangeIndex(start=0, stop=50000, step=1)

In [42]:
import tensorflow as tf


# Creating Tokeniser
tkn = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    oov_token="{OOV}"
)

# Tokenising Train Data
tkn.fit_on_texts(X)

In [43]:
len(tkn.word_index)

124253

In [44]:
X = tkn.texts_to_sequences(X)
X = tf.keras.preprocessing.sequence.pad_sequences(X,maxlen=200)

In [45]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((40000, 200), (10000, 200), (40000,), (10000,))

In [48]:
inputs = Input(shape=(None,), dtype="int32")

# Embed each integer in a 128-dimensional vector
x = Embedding(124254, 200)(inputs)

# Add 2 bidirectional LSTMs
x = LSTM(30,return_sequences=True)(x)
x = LSTM(30,return_sequences=True)(x)
x = LSTM(20,return_sequences=True)(x)
x = LSTM(20,return_sequences=True)(x)
x = LSTM(10,return_sequences=True)(x)
x = LSTM(10)(x)

# Add a classifier
outputs = Dense(1, activation="sigmoid")(x)
model = Model(inputs, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 200)         24850800  
_________________________________________________________________
lstm_18 (LSTM)               (None, None, 30)          27720     
_________________________________________________________________
lstm_19 (LSTM)               (None, None, 30)          7320      
_________________________________________________________________
lstm_20 (LSTM)               (None, None, 20)          4080      
_________________________________________________________________
lstm_21 (LSTM)               (None, None, 20)          3280      
_________________________________________________________________
lstm_22 (LSTM)               (None, None, 10)          1240

In [49]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=128, epochs=2, validation_data=(x_test, y_test))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
25600/40000 [==================>...........] - ETA: 5:26 - loss: 0.5070 - acc: 0.7573

AlreadyExistsError: Resource __per_step_297/Adam_4/gradients/lstm_19/while/ReadVariableOp/Enter_grad/ArithmeticOptimizer/AddOpsRewrite_Add/tmp_var/N10tensorflow19TemporaryVariableOp6TmpVarE
	 [[{{node Adam_4/gradients/lstm_19/while/ReadVariableOp/Enter_grad/ArithmeticOptimizer/AddOpsRewrite_Add/tmp_var}}]]